# Skeleton-Based Online Action Prediction Using Scale Selection Network
### A reimplementation in Keras

In [1]:
%matplotlib inline

# libraries for data access
import glob, os
import gzip
from numpy.random import permutation
from numpy import load, empty, ones
from cv2 import resize as imresize
import h5py

# libraries for computation
import keras
import numpy as np

#libraries for plotting
from matplotlib import pyplot as plt
import matplotlib.animation as animation
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128
from IPython.display import HTML



Using TensorFlow backend.


### Data

The dataset consists of a number of zip files where each file contains a multimodal sequence of actions. Included are : 
 - a video of the action
 - the extracted skeleton data aligned with the video
 - a foreground-background mask
 - a depth map

In [14]:
gestureSample = ChalearnLAPSample_wudi.GestureSample("./data/Train/Sample0001.zip")
fig, ax = plt.subplots(figsize=(25, 25))
im = ax.imshow(gestureSample.getSkeletonImage(1), animated=True)
def animate(i):
    img=gestureSample.getComposedFrame(i+1)
    im.set_array(img)
    return im,

anim = animation.FuncAnimation(fig, animate, interval=50, frames=100, repeat_delay=1000, blit=True)
HTML(anim.to_jshtml())

(480, 640, 3)


/usr/local/lib/python3.5/dist-packages/matplotlib/font_manager.py:1241: UserWarning: findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7fdcec127bf8> (for post_execute):


KeyboardInterrupt: 

In [2]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, path, window_size=100, batch_size=20, shuffle=True, feature='skeleton', yieldLabels=True, group='training'):
        'Initialization'
        self.batch_size = batch_size
        self.yieldLabels = yieldLabels
        self.window_size = window_size
        self.group = group
        self.path = path
        with h5py.File(self.path,'r') as file:
            self.list_IDs = list(range(file[self.group]['range'].shape[0]))
        self.shuffle = shuffle
        self.feature = feature
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'
        with h5py.File(self.path,'r') as file:
            # Initialization
            X = np.empty((self.batch_size, self.window_size, *file[self.group][self.feature][0].shape))
            y = np.empty((self.batch_size, self.window_size), dtype=int)

            # Generate data
            for i, ID in enumerate(list_IDs_temp):
                # Store sample
                X_entry = np.empty((self.window_size, *file[self.group][self.feature][0].shape))
                start, end = file[self.group]['range'][ID]
                X_timeseries = file[self.group][self.feature][start:end][:self.window_size]
                X_entry[:X_timeseries.shape[0]] = X_timeseries
                X[i,] = X_entry

                # Store class
                Y_entry = np.empty((self.window_size))
                Y_timeseries = file[self.group]['label'][start:end][:self.window_size]
                Y_entry[:Y_timeseries.shape[0]] = Y_timeseries
                y[i,] = Y_entry

            return X, y if self.yieldLabels else X

In [10]:
train_gen = DataGenerator('./data/dataset.hdf5')
#test_gen = DataGenerator('./data/dataset.hdf5', group='validation')

In [15]:
train_gen[166]

(array([[[[ 4.18779999e-002, -1.46750003e-001,  1.92410004e+000, ...,
            4.59790006e-002,  3.19000000e+002,  2.98000000e+002],
          [ 4.16520014e-002, -7.97120035e-002,  1.96050000e+000, ...,
           -1.01600001e-002,  3.18000000e+002,  2.79000000e+002],
          [ 3.95930000e-002,  2.73470014e-001,  1.93460000e+000, ...,
           -2.34899996e-003,  3.18000000e+002,  1.83000000e+002],
          ...,
          [-7.94809982e-002, -5.65289974e-001,  1.95620000e+000, ...,
           -3.18610013e-001,  2.86000000e+002,  4.10000000e+002],
          [-1.30469993e-001, -8.78700018e-001,  2.02900004e+000, ...,
            5.24949981e-003,  2.72000000e+002,  4.89000000e+002],
          [-1.32980004e-001, -9.37950015e-001,  1.95710003e+000, ...,
            1.30610000e-002,  2.67000000e+002,  5.15000000e+002]],
 
         [[ 4.21670005e-002, -1.46770000e-001,  1.92120004e+000, ...,
            4.32149991e-002,  3.19000000e+002,  2.98000000e+002],
          [ 4.18369994e-002, -

In [16]:
from models.utils.utils import generate_results_csv
from models.utils.utils import transform_labels
from models.utils.utils import create_directory
from models.utils.utils import read_dataset
from models.utils.utils import transform_mts_to_ucr_format
from models.utils.utils import visualize_filter
from models.utils.utils import viz_for_survey_paper
from models.utils.utils import viz_cam

import numpy as np
import sys
import sklearn 
#############randome seed:
seed = 50
np.random.seed(seed)
#TensorFlow has its own random number generator
from tensorflow import set_random_seed
set_random_seed(seed)
####################

def fit_classifier(classifier_name, train_gen, test_gen, output_directory): 
    input_shape = train_gen[0][0].shape[1:]
    classifier = create_classifier(classifier_name,input_shape, 100, output_directory)

    classifier.fit(train_gen,test_gen)

def create_classifier(classifier_name, input_shape, nb_classes, output_directory, verbose = False):
    if classifier_name=='fcn': 
        from models.classifiers import fcn        
        return fcn.Classifier_FCN(output_directory,input_shape, nb_classes, verbose)
    if classifier_name=='mlp':
        from  models.classifiers import  mlp 
        return mlp.Classifier_MLP(output_directory,input_shape, nb_classes, verbose)
    if classifier_name=='resnet':
        from  models.classifiers import resnet 
        return resnet.Classifier_RESNET(output_directory,input_shape, nb_classes, verbose)

      ########################################
    if classifier_name=='cnn': # Time-CNN
        from models.classifiers import cnn
        return cnn.Classifier_CNN(output_directory,input_shape, nb_classes, verbose)

############################################### main 

# change this directory for your machine
root_dir =r'./'
classifier_name='mlp'#'fcn'
itr = '_itr_8'
if itr == '_itr_0': 
    itr = ''

output_directory = root_dir+'/results/'+classifier_name+'/'+itr+'/'+'/'

create_directory(output_directory)


import time
start_time = time.time()

fit_classifier(classifier_name, train_gen, [], output_directory)
end_time = time.time()
print('Elapsed time:', end_time-start_time)
print (classifier_name)
print('DONE')

Epoch 1/100


OSError: Unable to open file (unable to open file: name = './data/dataset.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)